This code defines a ResNet50 architecture using TensorFlow's Keras API. ResNet50 is a convolutional neural network model that consists of 50 layers, known for its effectiveness in image classification tasks.

Here's a breakdown of the code:

1. **Imports**: The code imports necessary modules from TensorFlow, including `tf` for TensorFlow itself and specific modules like `Model` and various layers from `tensorflow.keras.models`.

2. **Identity Block Function**: `identity_block` function defines a block in the ResNet architecture where the input tensor is passed through a series of convolutional layers, with the addition of the original input tensor to the final output. This function is used for residual connections within the blocks.

3. **Convolutional Block Function**: `conv_block` function defines a block similar to `identity_block`, but with a convolutional layer with stride 2 at the beginning. This function is used when the input and output dimensions of the block are different.

4. **ResNet50 Function**: `resnet50` function constructs the entire ResNet50 architecture. It starts with an input layer followed by a convolutional layer and max-pooling layer. Then, it builds several convolutional blocks and identity blocks, forming the main body of the network. Finally, it adds a global average pooling layer and a fully connected layer with softmax activation for classification.

5. **Model Creation**: The ResNet50 model is created by calling the `resnet50` function.

6. **Model Summary**: The summary of the model, showing the layers, output shapes, and number of parameters, is printed out.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, GlobalAveragePooling2D, Dense

In [2]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = f'res{stage}_{block}_branch'
    bn_name_base = f'bn{stage}_{block}_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)

    x = Add()([x, input_tensor])
    x = Activation('relu')(x)
    return x

In [3]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    conv_name_base = f'res{stage}_{block}_branch'
    bn_name_base = f'bn{stage}_{block}_branch'

    x = Conv2D(filters1, (1, 1), strides=strides, name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(name=bn_name_base + '1')(shortcut)

    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x


In [4]:
def resnet50(input_shape=(256, 256, 3), num_classes=6):
    inputs = Input(shape=input_shape)

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(inputs)
    x = BatchNormalization(name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(num_classes, activation='softmax', name='fc')(x)

    model = Model(inputs, x, name='resnet50')
    return model

In [5]:
# Create the ResNet50 model
model = resnet50()
model.save('resnet.keras')